In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
import mysql.connector
from mysql.connector import Error

def main():
    # Database configuration (update with your credentials)
    config = {
        'host': 'localhost',
        'user': 'root',
        'password': 'mango4673',
        'database': 'anl503_eca'
    }
    
    # CSV file path
    csv_file_path = 'ECA_data_raw.csv'
    
    try:
        # Step 1: Read CSV file
        with open(csv_file_path, 'r', encoding='utf-8') as file:
            csv_reader = csv.DictReader(file)
            headers = csv_reader.fieldnames
            data = [row for row in csv_reader]
            
        # Step 2: Connect to MySQL
        conn = mysql.connector.connect(**config)
        cursor = conn.cursor()
        
        # Step 3: Create table if not exists
        create_table_query = (
            f"CREATE TABLE IF NOT EXISTS ECA_data_raw ("
            + ", ".join([f"`{header}` VARCHAR(255)" for header in headers])
            + ")"
        )
        cursor.execute(create_table_query)
        
        # Step 4: Prepare and execute insert query
        columns = ", ".join([f"`{header}`" for header in headers])
        placeholders = ", ".join(['%s'] * len(headers))
        insert_query = f"INSERT INTO ECA_data_raw ({columns}) VALUES ({placeholders})"
        
        # Convert data rows to tuples in the correct order
        values = []
        for row in data:
            row_values = [row[header] for header in headers]
            values.append(tuple(row_values))
            
        cursor.executemany(insert_query, values)
        conn.commit()
        
        print(f"Successfully imported {len(data)} rows into ECA_data_raw.")
        
    except FileNotFoundError:
        print(f"Error: CSV file not found at {csv_file_path}")
    except Error as e:
        print(f"MySQL Error: {e}")
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection closed.")

if __name__ == "__main__":
    main()

Successfully imported 6095 rows into ECA_data_raw.
MySQL connection closed.
